# Part 1. Running the simulations

It is common for a study to have to run few similar simulations with different values of some parameters. With Fluidsim, we usually do that with a Python script with arguments.

In [1]:
import shutil
import subprocess
from pathlib import Path

from IPython.display import Markdown as md

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.rc("figure", dpi=80)

In [3]:
import ipynbname

nb_path = ipynbname.path()
path_this_dir = nb_path.parent
script_run_simul = path_this_dir / "run_simul.py"
assert script_run_simul.exists()
script_submit_simul = path_this_dir / "submit_simul.py"
assert script_submit_simul.exists()

## One simulation script using argparse for the input parameters

It is convenient to write one script with input parameters. The simplest solution is to use the standard library module `argparse`.

We can also in this script handle the restart of simulations. However, here we keep it simple and do not care about restarting.

So here is our script "run_simul.py", which is a slightly simplified version of a script used for a real study.

In [4]:
with open(script_run_simul) as file:
    code = file.read()

md("```Python\n" + code + "```")

```Python
from math import pi
import argparse

from fluiddyn.util import mpi

from fluidsim.solvers.ns3d.strat.solver import Simul
from fluidsim.base.forcing.milestone import PeriodicUniform

from fluidsim.extend_simul.spatial_means_regions_milestone import (
    SpatialMeansRegions,
)
from fluidsim.extend_simul import extend_simul_class

Simul = extend_simul_class(Simul, SpatialMeansRegions)

parser = argparse.ArgumentParser()
parser.add_argument("-N", type=float, default=0.5, help="Brunt–Väisälä frequency")

parser.add_argument(
    "-D", "--diameter", type=float, default=0.25, help="Diameter of the cylinders"
)

parser.add_argument(
    "-s",
    "--speed",
    type=float,
    default=0.1,
    help="Maximum speed of the cylinders",
)

parser.add_argument(
    "-nc", "--number_cylinders", type=int, default=3, help="Number of cylinders"
)

parser.add_argument(
    "-nypc",
    "--ny_per_cylinder",
    type=int,
    default=60,
    help="Number of numerical points for one cylinder",
)

parser.add_argument(
    "-np",
    "--n_periods",
    type=float,
    default=5,
    help="Number of periods",
)

parser.add_argument(
    "-oi",
    "--only_init",
    action="store_true",
    help="Only run initialization phase",
)

parser.add_argument(
    "-wbl",
    "--width_boundary_layers",
    type=float,
    default=0.02,
    help="width boundary layers",
)


def main(args):

    params = Simul.create_default_params()

    diameter = args.diameter  # m
    speed = args.speed  # m/s
    params.N = args.N  # rad/s

    mesh = 3 * diameter
    number_cylinders = args.number_cylinders

    ly = params.oper.Ly = mesh * number_cylinders
    lx = params.oper.Lx = 1.5 * 3
    ny = params.oper.ny = args.ny_per_cylinder * number_cylinders

    nx_float = ny * lx / ly
    nx = params.oper.nx = round(nx_float)
    assert nx == nx_float

    dx = lx / nx
    mpi.printby0(f"{dx = }")

    lz = params.oper.Lz = mesh
    params.oper.nz = round(lz / dx)

    params.oper.coef_dealiasing = 0.8
    params.oper.NO_SHEAR_MODES = True
    params.no_vz_kz0 = True

    params.forcing.enable = True
    params.forcing.type = "milestone"
    params.forcing.milestone.nx_max = min(
        nx, round(16 * number_cylinders * nx / ny)
    )
    mpi.printby0(f"{params.forcing.milestone.nx_max = }")

    objects = params.forcing.milestone.objects

    objects.number = number_cylinders
    objects.diameter = diameter
    objects.width_boundary_layers = args.width_boundary_layers
    assert objects.width_boundary_layers < diameter / 4

    movement = params.forcing.milestone.movement

    movement.type = "periodic_uniform"
    movement.periodic_uniform.length = lx - 2 * diameter
    movement.periodic_uniform.length_acc = 0.25
    movement.periodic_uniform.speed = speed

    params.init_fields.type = "noise"
    params.init_fields.noise.velo_max = 5e-3

    movement = PeriodicUniform(
        speed,
        movement.periodic_uniform.length,
        movement.periodic_uniform.length_acc,
        lx,
    )

    params.time_stepping.t_end = movement.period * args.n_periods
    params.time_stepping.deltat_max = 0.04 * diameter / speed
    mpi.printby0(f"{params.time_stepping.deltat_max = }")

    params.nu_2 = 1e-6

    epsilon_eval = 0.02 * speed ** 3 / mesh
    eta_elav = (params.nu_2 ** 3 / epsilon_eval) ** (1 / 4)

    kmax = params.oper.coef_dealiasing * pi / dx
    eta_kmax = 2 * pi / kmax
    nu_2_needed = (epsilon_eval * eta_kmax ** 4) ** (1 / 3)

    mpi.printby0("eta_elav * kmax:", eta_elav * kmax)

    freq_nu4 = 0.5 * (nu_2_needed - params.nu_2) * kmax ** 2

    mpi.printby0("freq_nu4", freq_nu4)
    mpi.printby0("freq_nu4 / freq_nu2", freq_nu4 / (params.nu_2 * kmax ** 2))

    params.nu_4 = freq_nu4 / kmax ** 4

    params.output.sub_directory = "tutorial_parametric_study"
    if nx > 500:
        params.output.periods_print.print_stdout = movement.period / 50.0
    else:
        params.output.periods_print.print_stdout = movement.period / 20.0

    periods_save = params.output.periods_save
    periods_save.phys_fields = movement.period / 10.0
    periods_save.spatial_means = movement.period / 1000.0
    periods_save.spatial_means_regions = movement.period / 1000.0
    periods_save.spect_energy_budg = movement.period / 50.0
    periods_save.spectra = movement.period / 100.0
    periods_save.spatiotemporal_spectra = 2 * pi / params.N / 4

    params.output.spatial_means_regions.xmin = [0, 0.1, 0.4, 0.7]
    params.output.spatial_means_regions.xmax = [1, 0.3, 0.6, 0.9]

    spatiotemporal_spectra = params.output.spatiotemporal_spectra
    spatiotemporal_spectra.file_max_size = 20.0
    spatiotemporal_spectra.probes_region = (10 * lx / ly, 10, 10)

    params.output.spectra.kzkh_periodicity = 2

    sim = Simul(params)

    if not args.only_init:
        sim.time_stepping.start()

    return params, sim


if __name__ == "__main__":
    args = parser.parse_args()
    mpi.printby0(args)

    params, sim = main(args)
```

## One script to submit jobs on a cluster

Usually for parametric studies, we need to launch several simulations with different parameters. There are great tools to automate such scientific workflows (for example [execo](https://pypi.org/project/execo/)) but here, we keep it simple and just show a small script to launch simulations.

Since modifying manually launching scripts in Bash is very error prone, we automate the process with the [fluiddyn.clusters](https://fluiddyn.readthedocs.io/en/latest/generated/fluiddyn.clusters.html) package.

In [5]:
with open(script_submit_simul) as file:
    code = file.read()

md("```Python\n" + code + "```")

```Python
from fluiddyn.clusters.legi import Calcul8 as Cluster

cluster = Cluster()

cluster.commands_setting_env = [
    "source /etc/profile",
    ". $HOME/miniconda3/etc/profile.d/conda.sh",
    "conda activate env_fluidsim",
    "export FLUIDSIM_PATH=/fsnet/project/watu/2020/20MILESTONE",
]

diameters = [0.25, 0.5]
velocities = [0.05, 0.1, 0.2]

for diameter in diameters:
    for speed in velocities:
        cluster.submit_script(
            f"run_simul.py -D {diameter} -s {speed}",
            name_run="milestone",
            nb_cores_per_node=10,
            nb_mpi_processes=10,
            omp_num_threads=1,
            ask=False,
        )
```

In [6]:
import fluidsim as fls

path_dir_data = Path(fls.FLUIDSIM_PATH) / "tutorial_parametric_study"
shutil.rmtree(path_dir_data, ignore_errors=True)

For this tutorial, we don't submit the simulations on a cluster but just run 3 simulations sequentially by calling directly `mpirun`.

In [7]:
longer_simulations = True
if longer_simulations:
    ny_per_cylinder = 48
    speeds = [0.01, 0.02, 0.04, 0.06, 0.08]
else:
    ny_per_cylinder = 24
    speeds = [0.08]

In [8]:
logs = []
for isim, speed in enumerate(speeds):
    print("-" * 79 + f"\nSimulation {isim}")
    process = subprocess.run(
        (
            f"mpirun -np 2 python {script_run_simul} "
            f"-N 0.2 --diameter 0.5 --speed {speed} "
            f"--ny_per_cylinder {ny_per_cylinder} "
            "--number_cylinders 3 --width_boundary_layers 0.1 --n_periods 2"
        ).split(),
        stdout=subprocess.PIPE,
        stderr=subprocess.DEVNULL,  # hiding irrelevant warnings
        text=True,
    )
    lines = process.stdout.split("\n")
    print("\n".join(lines[:50]) + "\n[...]\n" + "\n".join(lines[-10:]))

-------------------------------------------------------------------------------
Simulation 0
Namespace(N=0.2, diameter=0.5, speed=0.01, number_cylinders=3, ny_per_cylinder=48, n_periods=2.0, only_init=False, width_boundary_layers=0.1)
dx = 0.03125
params.forcing.milestone.nx_max = 48
params.time_stepping.deltat_max = 2.0
eta_elav * kmax: 0.23667650850121094
freq_nu4 0.25288935238750754
freq_nu4 / freq_nu2 39.09766924201435

--------
nb_proc: 2
N0 = 48 ; N1 = 144 ; N2 = 144
Initialization (FFT3DMPIWithFFTWMPI3D) done in 0.951517 s
*************************************
Program fluidsim
sim:                <class 'fluidsim_core.extend_simul.extend_simul_class.<locals>.NewSimul'>
sim.output:         <class 'fluidsim.solvers.ns3d.strat.output.Output'>
sim.oper:           <class 'fluidsim.operators.operators3d.OperatorsPseudoSpectral3D'>
sim.state:          <class 'fluidsim.solvers.ns3d.strat.state.StateNS3DStrat'>
sim.time_stepping:  <class 'fluidsim.solvers.ns3d.time_stepping.TimeSteppingP

The data created during the simulations should be in `path_dir_data`. Let's have a look:

In [9]:
path_runs = sorted(path_dir_data.glob("*"), key=lambda p: p.name)
[p.name for p in path_runs]

['ns3d.strat_144x144x48_V4.5x4.5x1.5_N0.2_Lf3.5_U0.01_D0.5_2021-10-05_15-35-47',
 'ns3d.strat_144x144x48_V4.5x4.5x1.5_N0.2_Lf3.5_U0.02_D0.5_2021-10-05_16-12-36',
 'ns3d.strat_144x144x48_V4.5x4.5x1.5_N0.2_Lf3.5_U0.04_D0.5_2021-10-05_16-49-31',
 'ns3d.strat_144x144x48_V4.5x4.5x1.5_N0.2_Lf3.5_U0.06_D0.5_2021-10-05_17-25-48',
 'ns3d.strat_144x144x48_V4.5x4.5x1.5_N0.2_Lf3.5_U0.08_D0.5_2021-10-05_18-01-23']

We see that the directory names are understandable by humans and tell us about the parameters of the simulations.